In [ ]:
import pandas as pd
import numpy as np
import numpy_financial as npf
import random
import math
import datetime as dt
import arrow

from __future__ import annotations

In [37]:
from __future__ import annotations

import datetime as dt
import math

import numpy as np
import numpy_financial as npf
import pandas as pd


class PrepaymentException(Exception):
    pass

class Loan:
    """
    ## Loan class to create instances of installment loans.

    To be initialized with the following parameters:

    1. `loan_amount:float` - [Required] The original loan amount (principal)
    disbursed
    on the loan date.
    2. `int_rate:float` - [Required] original rate of interest applicable on
    the principal outstanding.
    3. `term_in_months:float` - [Required] The original term of the loan per
    schedule.
    4. `loan_dt:str` - Date of loan disbursement.
    5. `freq:str` - [Optional; default value:'M'] Frequency of repayment.
    Monthly, Quarterly etc. Valid values can be accessed via the class variable
    `Loan.valid_pmt_freq`.

        >    Valid inputs to the frequency variable include the below:

            - 'W': 'Weekly payments'
            - '2W': 'Fortnightly payments'
            - 'M': 'Monthly payments'
            - 'BM': 'Bi-monthly payments'
            - 'Q': 'Quarterly payments'
            - 'H': 'Semi-annual payments'
            - 'Y': 'Annual payments'

    6. `fees:float` - [Optional; default value: 0.0] Origination fees charged
    at the time of booking, expressed as a % of original loan amount.
    7. `addl_pmts: dict` - [Optional; default value: None] A dictionary
    containing all additional payments made over and above the scheduled
    payments for the loan obligation.
    8. `segment:str` - [Optional; default value: 'c'] The approx risk
    category of the loan. Broadly mapped to six FICO_score groups.
    Configurable via config.yaml for categories'
    9. `channel:str` - [Optional; default value: 'M'] Indicator variable to
    identify if the loan was booked through a free channel or a paid channel.

    Apart from the initialization parameters mentioned above, the loan
    object also has the following attributes:

    1. `pmt: float` - Based on the initial parameters of the loan,
    the attribute reflects the original equated installment amount.
    2.  `original_cfs: pd.DataFrame` - A dataframe with the original
    schedule of cashflows based on the loan parameters is returned. This
    does not include the additional payments made.
    3. `updated_cfs: pd.DataFrame` - A dataframe with the modified
    schedule of cashflows based on the loan parameters is returned. This
    considers the additional payments made.
    4. `fully_prepaid: int` - A flag like parameter to indicate of the loan
    was fully pre-paid. In case the loan in fully pre-paid, no additional
    payments can be specified and hence there cannot be further
    modifications to the cashflows.

    >**Examples:**
    ```python
    import pyloans as pyl

    l1 = pyl.Loan.Loan(loan_amt=20000, interest_rate=0.1099, term_in_months=60,
                        loan_dt="2022-12-12", freq="M",
                        addl_pmts={3: 200, 4: 300, 5: 400, 6: 500,
                        },
                    )

    # Get the original schedule of cashflows without considering additional
    # payments, if any:

    l1.original_cfs

    # Get the modified schedule of cashflows considering addition payments,
    # if any:

    l1.updated_cfs
    ```
        """

    valid_pmt_freq = {
        'W': 'Weekly payments', '2W': 'Fortnightly payments',
        'M': 'Monthly payments', 'BM': 'Bi-monthly payments',
        'Q': 'Quarterly payments', 'H': 'Semi-annual payments',
        'Y': 'Annual payments',
    }
    freq_offset = {
        'W': pd.DateOffset(days=7, months=0),
        '2W': pd.DateOffset(days=14, months=0),
        'M': pd.DateOffset(days=0, months=1),
        'BM': pd.DateOffset(days=0, months=2),
        'Q': 'Q', 'H': '2Q', 'Y': 'Y',
    }
    period_to_months = {
        'W': 7.0 / 30, '2W': 14.0 / 30,
        'M': 1, 'BM': 2, 'Q': 3, 'H': 6, 'Y': 12,
    }
    cols = [
        'dates', 'period', 'opening_principal', 'opening_accrued_interest',
        'current_period_interest', 'interest_pmt', 'principal_pmt',
        'additional_pmt', 'total_pmt', 'closing_principal',
    ]

    def __init__(
            self, loan_amt: float, interest_rate: float, term_in_months: float,
            loan_dt: str, freq: str = 'M', fees_pct: float = 0.0,
            addl_pmts: dict | None = None, segment: str = 'c', channel: str
            = 'free',
    ):
        self.loan_amt = loan_amt
        self.interest_rate = interest_rate
        self.term_in_months = term_in_months
        self.loan_dt = dt.datetime.strptime(loan_dt, '%Y-%m-%d')
        self.freq = freq
        self.fees_pct = fees_pct
        self.addl_pmts = addl_pmts if addl_pmts else {}
        self.segment = segment
        self.channel = channel
        self._offset = self.freq_offset[self.freq]
        self._periods = math.ceil(
            self.term_in_months / self.period_to_months[self.freq],
        )
        self._period_interest_rate = self.interest_rate * self \
            .period_to_months[self.freq] / 12
        self.pmt = -npf.pmt(
            self._period_interest_rate,
            self._periods, self.loan_amt,
        )
        self.fully_prepaid = 0
        self.original_cfs = self.get_org_cfs()
        self.updated_cfs = self.get_org_cfs()
        self.updated_cfs = self._get_mod_cfs()

    def get_org_cfs(self) -> pd.DataFrame:
        """Method to get the original scheduled of cashflows for a given loan.
        For monthly frequency (most common), it assumes that the dues date are
        on the same day of the month every month.
        >**Usage**:
            ```python
            Loan_obj.get_schedule()
            ```
            """
        df = pd.DataFrame()
        df['dates'] = pd.Series(
            pd.date_range(
                self.loan_dt, freq=self._offset, periods=self._periods + 1,
            ),
        ).shift(-1).dropna()
        df['period'] = df.index + 1
        df['interest_pmt'] = - \
            npf.ipmt(
                self._period_interest_rate,
                df['period'], self._periods, self.loan_amt,
            )
        df['principal_pmt'] = - \
            npf.ppmt(
                self._period_interest_rate,
                df['period'], self._periods, self.loan_amt,
            )
        df['closing_principal'] = self.loan_amt - df['principal_pmt'].cumsum()
        df['opening_principal'] = df['closing_principal'].shift(
            1,
        ).fillna(self.loan_amt)
        df['additional_pmt'] = pd.Series(
            self.addl_pmts,
            index=df.index,
        ).fillna(0)
        df['current_period_interest'] = df['interest_pmt']
        df['opening_accrued_interest'] = 0
        df['closing_accrued_interest'] = 0
        df['additional_pmt'] = 0
        df['total_pmt'] = df['interest_pmt'] + df['principal_pmt'] + \
            df['additional_pmt']
        df = df[self.cols]
        return df

    def _wal(self, _df: pd.DataFrame) -> float:
        return ((_df['opening_principal'] - _df['closing_principal']) *
                _df['period']).sum() * \
            self.period_to_months[self.freq] / self.loan_amt

    @property
    def org_wal(self) -> float:
        """Returns the weighted average life of the loan (in months) based on
        a given cashflow schedule.
        The [WAL](https://en.wikipedia.org/wiki/Weighted-average_life) of the
        loan can be defined as the average number of months it takes for the
        principal of the loan to be repaid, if the borrower repays by the
        original schedule.
        >**Usage**:
            ```python
            Loan_obj.org_wal
            ```
        """
        return self._wal(self.get_org_cfs())

    @property
    def org_apr(self) -> float:
        """Returns the Annual percentage rate (APR) of the loan based on the
        original cashflow schedule.
        The [APR](https://en.wikipedia.org/wiki/Annual_percentage_rate) of the
        loan can be defined as the total financial cost of the loan (
        including fees) divided by the WAL of the loan.
        >**Usage**:
            ```python
            Loan_obj.org_apr
            ```
        """
        return self.interest_rate + (self.fees_pct / (self.org_wal / 12))

    def _merge_addl_pmt(self, addl_pmt_update: dict) -> dict:
        keys = set(list(self.addl_pmts.keys()) + list(addl_pmt_update.keys()))
        return {
            k: self.addl_pmts.get(k, 0.0) + addl_pmt_update.get(k, 0.0)
            for k in keys
        }

    def _get_mod_cfs(self) -> pd.dataFrame:
        if self.fully_prepaid == 1:
            return self.updated_cfs
        else:
            df = self.updated_cfs
            if self.addl_pmts:
                cl_p = self.loan_amt
                cl_ai = 0  # accrued interest for 1st period
                df['additional_pmt'] = pd.Series(
                    {k-1: v for k, v in self.addl_pmts.items()},
                    index=df.index,
                ).fillna(0)
                for idx, row in df.iterrows():
                    row.loc['opening_principal'] = cl_p
                    row.loc['opening_accrued_interest'] = cl_ai
                    row.loc['current_period_interest'] = \
                        row.loc['opening_principal'] * \
                        self._period_interest_rate
                    row.loc['total_pmt'] = min(
                        row.loc['interest_pmt'] +
                        row.loc['principal_pmt'] +
                        row.loc['additional_pmt'],
                        row.loc['opening_principal'] +
                        row.loc['opening_accrued_interest'] +
                        row.loc['current_period_interest'],
                    )
                    row.loc['closing_accrued_interest'] = \
                        max(
                            0,
                            row.loc['opening_accrued_interest'] +
                            row.loc['current_period_interest'] -
                            row.loc['total_pmt'],
                        )
                    cl_ai = row.loc['closing_accrued_interest']
                    row.loc['closing_principal'] = \
                        max(
                            0,
                            row.loc['opening_principal'] +
                            row.loc['opening_accrued_interest'] +
                            row.loc['current_period_interest'] -
                            row.loc['total_pmt'],
                        )
                    cl_p = row.loc['closing_principal']
                    df.loc[idx, self.cols] = row.loc[self.cols]
            else:
                df = self.get_org_cfs()
            self.updated_cfs = df
            return self.updated_cfs

    @property
    def mod_wal(self) -> float:
        """
        Returns the WAL of the loan object based on the additional payments
        provided. The WAL is based on the `updated_cfs` attribute of the loan
        object.
        > **Usage:**
            ```python
            l1.mod_wal
            ```
        """
        return self._wal(self.updated_cfs)

    @property
    def mod_apr(self) -> float:
        """
        Returns the APR of the loan object based on the additional payments
        provided. The APR is based on the `updated_cfs` attribute of the loan
        object.
        > **Usage:**
            ```python
            l1.mod_apr
            ```
        """
        return self.interest_rate + (self.fees_pct / (self.mod_wal / 12))

    def _maturity(self) -> np.int64:
        _cfs = self.updated_cfs
        # numpy_financial precision issue
        return _cfs[_cfs.closing_principal <= 1e-9].period.min()

    @property
    def org_maturity_period(self):
        """
        Returns the original maturity in periods, which is same as the
        term_in_months, converted to the corresponding periods based on the
        payment frequency.
        > **Usage:**
            ```python
            l1.org_maturity_period()
            ```
        """
        return self._periods

    @property
    def mod_maturity_period(self):
        """
        Returns the modified maturity after considering additional payments,
        if any.
        > **Usage:**
            ```python
            l1.org_maturity_period()
            ```
        """
        return self._maturity()

    def prepay_fully(self, period: int) -> pd.DataFrame:
        """
        The method checks the status of the loan if it is already fully
        pre-paid. If the loan is already fully pre-paid, we raise an
        exception notifying the same.
        Else, the outstanding principal amount is considered as the
        additional payment amount and the closing balance of the loan is
        zero-ed out in the period specified as the input. The `fully_prepaid`
        flag is also set to 1.
        > **Usage:**
            ```python
            l1.org_maturity_period(6)
            ```
        """
        if self.fully_prepaid == 1:
            raise PrepaymentException('Loan is already pre-paid fully')
        else:
            df = self.updated_cfs
            prepay_amt = df.loc[period-1, 'closing_principal']
            self.addl_pmts = self._merge_addl_pmt({period: prepay_amt})
            self.updated_cfs = self._get_mod_cfs()
            self.fully_prepaid = 1
            return self.updated_cfs

    def update_addl_pmts(self, addl_pmt_update: dict) -> pd.DataFrame:
        """
        The method checks the status of the loan if it is already fully
        pre-paid. If the loan is already fully pre-paid, we raise an
        exception notifying the same.
        Else, the `addl_pmts` attribute of the loan object is merged to
        include the additional payment passed to the method.
        > **Usage:**
            ```python
            l1.update_addl_pmts({7:700})
            ```
        """
        if self.fully_prepaid == 1:
            raise PrepaymentException(
                'Loan already fully pre-paid. Cannot '
                'make additional payments',
            )
        elif self.addl_pmts:
            self.addl_pmts = self._merge_addl_pmt(addl_pmt_update)
            self.updated_cfs = self._get_mod_cfs()
            return self.updated_cfs
        else:
            self.addl_pmts = addl_pmt_update
            self.updated_cfs = self._get_mod_cfs()
            return self.updated_cfs

    def reset_addl_pmts(self) -> None:
        """
        Since the `update_addl_pmts` method merges the input with the
        existing attribute `addl_pmts` of the loan object. The method provides
        a way to reset the `addl_pmts` attribute in case of any errors.
        > **Usage:**
            ```python
            l1.reset_addl_pmts()
            ```
        """
        self.addl_pmts = {}
        self.fully_prepaid = 0
        self.updated_cfs = self._get_mod_cfs()


In [38]:
irt, trm, frq, fee, adp = (0.1099, 72, 'M', 0.05, {3: 200, 4: 300, 5: 400, 6: 500})
l1 = Loan(loan_amt=20000, interest_rate=irt, term_in_months=trm, loan_dt='2022-12-12',
                      freq=frq, fees_pct=fee, addl_pmts = adp)

In [39]:
#import tabulate
df = l1.original_cfs

In [48]:
l1.reset_addl_pmts()
# l1.prepay_fully(12)
# l1.reset_addl_pmts()
# l1.updated_cfs
l1.update_addl_pmts({3: 200, 4: 300, 5: 400, 6: 500})
print(l1.mod_maturity_period)
l1.update_addl_pmts({7: 700})
print(l1.mod_maturity_period)
l1.reset_addl_pmts()
print(l1.mod_maturity_period)

/tmp/ipykernel_236056/8062002.py:174: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  df['additional_pmt'] = pd.Series(
/tmp/ipykernel_236056/8062002.py:174: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  df['additional_pmt'] = pd.Series(


66
63
72


In [34]:
l1.original_cfs

,dates,period,opening_principal,opening_accrued_interest,current_period_interest,interest_pmt,principal_pmt,additional_pmt,total_pmt,closing_principal
0,2023-01-12,1,20000.000000,0,183.166667,183.166667,197.412483,0,380.57915,1.980259e+04
1,2023-02-12,2,19802.587517,0,181.358697,181.358697,199.220453,0,380.57915,1.960337e+04
2,2023-03-12,3,19603.367064,0,179.534170,179.534170,201.044980,0,380.57915,1.940232e+04
3,2023-04-12,4,19402.322084,0,177.692933,177.692933,202.886217,0,380.57915,1.919944e+04
4,2023-05-12,5,19199.435867,0,175.834833,175.834833,204.744317,0,380.57915,1.899469e+04
...,...,...,...,...,...,...,...,...,...,...
67,2028-08-12,68,1851.710796,0,16.958585,16.958585,363.620565,0,380.57915,1.488090e+03
68,2028-09-12,69,1488.090231,0,13.628426,13.628426,366.950724,0,380.57915,1.121140e+03
69,2028-10-12,70,1121.139507,0,10.267769,10.267769,370.311381,0,380.57915,7.508281e+02
70,2028-11-12,71,750.828127,0,6.876334,6.876334,373.702816,0,380.57915,3.771253e+02


In [13]:
# channel_prop = {'free':0.8, 'paid':0.2}
# channel_cost = {'free':0.0, 'paid':0.05}
# term_prop = {'36':0.7, '60':0.3}
# term_prem = {'36':0.0, '60':0.01} #reference term premium from treasury site
# fee = {'No Fees': 0, '5% fees': 0.05, '10% fees': 0.1}

# segment_ScoreMapping = {'f':'FICO:300-550', 'e':'FICO:550-650', 'd':'FICO:650-699',
#                 'c':'FICO:700-749', 'b':'FICO:750-799', 'a':'FICO:800-850'}

# segment_prop = {'f':{'300-550':8.4%}, 'e':{'550-650':16.3%}, 'd':{'650-699':12.5%},
#                 'c':{'700-749':16.4%}, 'b':{'750-799':23.1%}, 'a':{'800-850':23.3%}} #six segments tied to chargeoffs and prepayment rates
# chOff_mean = {'a':1%, 'b':1%, 'c':4.4%, 'd':8.9%, 'e':mean(22.5%, 15.8%), 'f':mean(28.4%, 41%)} # researchgate reference from 2000-2002
# #establish a relationship between chargeoff and prepayment rates or FICO and prepayment rates
# prepay_mean = {'a':45%, 'b':40%, 'c':35%, 'd':30%, 'e':25%, 'f':20%} # purely arbitrary


In [ ]:
#Generating the prepayment curve and the Loss curve

# Losses peak by 9 months an36 std of about 3 months for a 36 month loan = 9/36 = 0.25; var = 9 mthsq = 9/36^2 = 0.00695 
# prepayments peak by 12 months and std by 6 months for a 36 month loan = 12/36 = 0.33; var = 9 mthsq = 36/36^2 = 0.0278

# alpha = mean*((mean*(1-mean)/var)-1)
# beta = (1-mean)*((mean*(1-mean)/var)-1)

# alpha_co = 0.25*((0.25*(1-0.25)/0.00695)-1)
# beta_co = (1-0.25)*((0.25*(1-0.25)/0.00695)-1)

# alpha_ppay = 0.33*((0.33*(1-0.33)/0.0278)-1)
# beta_ppay = (1-0.33)*((0.33*(1-0.33)/0.0278)-1)


In [ ]:
# #sampling from a beta distribution
# x = math.floor(36*random.betavariate(alpha_co, beta_co))
# x

In [ ]:
# y = math.floor(36*random.betavariate(alpha_ppay, beta_ppay))
# y